In [ ]:
#| default_exp datasets.pipal

# PIPAL

> Building a `tf.data.Dataset` for Kadik10k.

In [ ]:
#| hide
import os; os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [ ]:
#| export
from pathlib import Path
from typing import List

import pandas as pd
import tensorflow as tf
import cv2

After setting up the path to the directory and loading the corresponding `.csv` file, we need to create a generator that will iterate over the dataframe, load and return a 3-tuple: `(Reference Image, Distorted Image, DMOS)`. When can the pass that generator into a `tf.data.Dataset.from_generator()` to build the `Dataset` object:

In [ ]:
#| export
class PIPAL():
    """Builder for the PIPAL dataset"""

    def __init__(self,
                 path, # Path to the root directory of the dataset.
                 exclude_imgs: List[int] = None, # Image ID's to exclude.
                 exclude_dist: List[int] = None, # Distortion ID's to exclude.
                 exclude_ints: List[int] = None, # Distortion Intensities ID's to exclude.
                 ):
        self.path_root = Path(path) if isinstance(path, str) else path
        self.path_csv = self.path_root / "image_pairs_mos.csv"
        self.path_ref = self.path_root / "Train_Ref"
        self.data = self.load_data(self.path_csv, exclude_imgs, exclude_dist, exclude_ints)

    @property
    def dataset(self):
        """tf.data.Dataset object built from the PIPAL dataset."""
        return tf.data.Dataset.from_generator(
                self.data_gen,
                output_signature=(
                    tf.TensorSpec(shape=(288, 288, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(288, 288, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(), dtype=tf.float32)
                )
            ) 

    def data_gen(self):
        """Dataset generator to build the tf.data.Dataset."""
        for i, row in self.data.iterrows():
            ref, dist, mos, directory = row.Reference, row.Distorted, row.MOS, row.Directory
            dist = cv2.imread(str(self.path_root/directory/dist))
            dist = cv2.cvtColor(dist, cv2.COLOR_BGR2RGB)/255.0
            ref = cv2.imread(str(self.path_ref/ref))
            ref = cv2.cvtColor(ref, cv2.COLOR_BGR2RGB)/255.0
            yield ref, dist, mos

    def load_data(self,
                  path,
                  exclude_imgs,
                  exclude_dist,
                  exclude_ints,
                  ):
        data = pd.read_csv(self.path_csv, index_col=0)
        data = data[~data.Reference_ID.isin(exclude_imgs)] if exclude_imgs is not None else data
        data = data[~data.Reference_ID.isin(exclude_dist)] if exclude_dist is not None else data
        data = data[~data.Reference_ID.isin(exclude_ints)] if exclude_ints is not None else data
        return data


In [ ]:
l = PIPAL(path = Path("/media/disk/databases/BBDD_video_image/Image_Quality/PIPAL"))

In [ ]:
l.data.head()

,Reference,Distorted,MOS,Directory
0,A0001.bmp,A0001_00_00.bmp,1520.0648,Distortion_1
1,A0001.bmp,A0001_00_01.bmp,1437.0798,Distortion_1
2,A0001.bmp,A0001_00_02.bmp,1546.0616,Distortion_1
3,A0001.bmp,A0001_00_03.bmp,1539.5688,Distortion_1
4,A0001.bmp,A0001_00_04.bmp,1411.7958,Distortion_1


In [ ]:
for a, b, c in l.dataset:
    break
assert a.shape == b.shape